In [1]:
import praw
import datetime
import json
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
import string
import pandas as pd
import collections
from pprint import pprint
from nltk.tokenize import word_tokenize, RegexpTokenizer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/bruce/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
file = open('reddit_info.json')
info = json.load(file)

In [3]:
#Basically logs into the bot? I think
def create_reddit_object():
    reddit = praw.Reddit(client_id = info['client_id'],
                        client_secret = info['client_secret'],
                        user_agent = info['user_agent'],
                        username = info['username'],
                        password = info['password'])
    return reddit

In [4]:

# 1000 posts may not be enough, so compile 1000 posts from each subreddit of choosing.
def compile_subreddits(subreddits):
    headlines = set()

    reddit = create_reddit_object()

    for i in range(len(subreddits)):
        subreddit = reddit.subreddit(subreddits[i])
        top_headlines_in_year = subreddit.top("year", limit=None)

        for submission in top_headlines_in_year:
            headlines.add(submission.title)
    
    return headlines



In [5]:

# Here I am compiling all posts from popular, news, all, politics, and worldnews subreddits.
all_headlines = compile_subreddits(["popular", "news", "all", "politics", "worldnews"])


In [11]:

''' Remove punctuation and digits from a string. '''
def _process_string(s):
    no_digits = []

    # Convert string to lowercase, remove punctuation.
    raw_string = s.lower()
    clean_string = raw_string.translate(str.maketrans('','', string.punctuation))

    # Remove all digits from string.
    for letter in clean_string:
        if not letter.isdigit():
            no_digits.append(letter)

    # Create the final string.
    result = ''.join(no_digits)

    return result

In [32]:
'''Find the most common word'''
def find_common_words():

    reddit = create_reddit_object()

    wordcounts = {}

    # Loop through the set with submission titles, remove stop words, and get a dictionary of word counts
    for submission in all_headlines:
        for word in _process_string(submission).split():
            if word not in stopwords.words('english'):
                if len(word) > 2:
                    if word not in wordcounts:
                        wordcounts[word] = 1
                    else:
                        wordcounts[word] += 1
    return wordcounts

word_counter = collections.Counter(find_common_words())
for word, count in word_counter.most_common(500):
    print(word, ":", count)

find_common_words()


988
